In [1]:
import requests
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
token="здесь должен быть токен"

In [3]:
group_name="vedomosti"
offset=0
count=100
all_posts=[]
all_comments=[]

In [4]:
offset=0
while offset<2100:
        req=requests.get('https://api.vk.com/method/wall.search',
               params={
                   'access_token': token,
                   'v': 5.131,
                   'domain': group_name,
                   'query': 'инфляция',
                   'count': count,
                   'offset': offset
                   
               })
        res=req.json()['response']['items']
        offset+=100
        time.sleep(1)
        all_posts.extend(res)

In [5]:
res=req.json()['response']['items']
all_posts.extend(res)
df = pd.json_normalize(all_posts)

In [6]:
df=df[["id", "owner_id", "date", "text", "comments.count"]]
df.rename(columns={'date': 'post_time','text':'post_text'}, inplace=True)
df=df[df['post_time']>=1630454400]
df=df[df['comments.count']!=0]
df

,id,owner_id,post_time,post_text,comments.count
0,1756398,-15548215,1674489612,Минюст предлагает новые условия содержания зак...,17
2,1754848,-15548215,1674455001,так индексировали меньше инфляции вот и не защ...,1
4,1754848,-15548215,1674433201,Регулярная индексация минимальных заработных п...,17
9,1746014,-15548215,1674079200,"JPMorgan, Bank of America, Citigroup и другие ...",12
12,1745087,-15548215,1674051018,А почему бы инфляцию не отменить очередным ука...,10
...,...,...,...,...,...
939,1319778,-15548215,1631149207,Экономические власти предложили стратегию защи...,5
941,1319678,-15548215,1631122219,«Нельзя постоянно печатать деньги и не платить...,19
944,1318306,-15548215,1630662129,То есть свою инфляцию они не видят?!!!!,3
945,1318148,-15548215,1630607413,Центральный банк включил в альтернативные сцен...,7


In [7]:
owner_id=-15548215
posts_id=list(df['id'])

In [8]:
for post_id in posts_id:
    req=requests.get('https://api.vk.com/method/wall.getComments',
               params={
                   'access_token': token,
                   'v': 5.131,
                   'owner_id': owner_id,
                   'post_id': post_id
               })
    time.sleep(1)
    res=req.json()['response']['items']
    all_comments.extend(res)

In [9]:
data = pd.json_normalize(all_comments)
data

,id,from_id,date,text,post_id,owner_id,parents_stack,thread.count,thread.items,thread.can_post,thread.show_reply_button,thread.groups_can_post,attachments,deleted
0,1756406,21333770,1674489846,"У нас в тюрьмах преступников то не осталось, о...",1756398.0,-15548215.0,[],1,[],True,True,True,NaN,NaN
1,1756412,20291756,1674489923,Пригожин и Путин срочно разгружают зоны от зэк...,1756398.0,-15548215.0,[],5,[],True,True,True,NaN,NaN
2,1756437,694295529,1674490691,Вот действительно видна забота государства о р...,1756398.0,-15548215.0,[],0,[],True,True,True,NaN,NaN
3,1756802,624037853,1674499271,Созывают что ли к лучшей жизни😅 реклама скоро ...,1756398.0,-15548215.0,[],0,[],True,True,True,NaN,NaN
4,1756811,2773473,1674499440,"А в больницах они не хотят улучшить питание, ч...",1756398.0,-15548215.0,[],4,[],True,True,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2594,1318042,112809857,1630582044,Скорее этими деньгами пенсионеры и пойдут голо...,1318012.0,-15548215.0,[],0,[],True,True,True,NaN,NaN
2595,1318050,481138434,1630583453,Либерастики и комуняки вечно страдающие за бед...,1318012.0,-15548215.0,[],0,[],True,True,True,NaN,NaN
2596,1318059,487401888,1630585691,Не голосую лет 30👍,1318012.0,-15548215.0,[],0,[],True,True,True,NaN,NaN
2597,1318061,113269503,1630585854,Не помогает Правительство - плохо. Помогает - ...,1318012.0,-15548215.0,[],0,[],True,True,True,NaN,NaN


In [10]:
data=data[["post_id", "text", "date"]]
data.rename(columns={'date': 'comment_time', 'id': 'comment_id'}, inplace=True)
data=data.dropna()

In [11]:
data['post_id'] = data['post_id'].astype(int)
data

,post_id,text,comment_time
0,1756398,"У нас в тюрьмах преступников то не осталось, о...",1674489846
1,1756398,Пригожин и Путин срочно разгружают зоны от зэк...,1674489923
2,1756398,Вот действительно видна забота государства о р...,1674490691
3,1756398,Созывают что ли к лучшей жизни😅 реклама скоро ...,1674499271
4,1756398,"А в больницах они не хотят улучшить питание, ч...",1674499440
...,...,...,...
2594,1318012,Скорее этими деньгами пенсионеры и пойдут голо...,1630582044
2595,1318012,Либерастики и комуняки вечно страдающие за бед...,1630583453
2596,1318012,Не голосую лет 30👍,1630585691
2597,1318012,Не помогает Правительство - плохо. Помогает - ...,1630585854


In [12]:
df.to_csv('vedomosti_posts.csv', index=False)

In [13]:
data.to_csv('vedomosti_comments.csv', index=False)